# [Pump it Up: Data Mining the Water Table](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/)
### Maher Deeb
Best score 0.8249 and best ranking is 42

## Abstract

The goal of this project is to build a predective model using data from Taarifa and the Tanzanian Ministry of Water to predict which pumps are functional, which need some repairs, and which don't work at all. Predict one of these three classes is based on a number of variables about what kind of pump is operating, when it was installed, and how it is managed. A smart understanding of which waterpoints will fail can improve maintenance operations and ensure that clean, potable water is available to communities across Tanzania.

## Table of content
    
    1. Abstract
    2. Table of content
    3. Introduction
    3. Data exploring
    4. Data cleaning
    5. Model training
    6. Predection
    
## Introduction

This project is a multi-class classification problem. The goal is to predict which pumps are functional, which need some repairs, and which don't work at all. To solve this problem the data will loaded and explored to address its qualilty and apply data cleaning method to get tidy data. The data will be used to train a classifier based on *Random Forests* method. Finally the model will be tested on a cross validation dataset before prediction the classes of the given testing dataset that should be submitted to the competition website.

## Data exploring:

As a first step the important libraries which are needed to load the datasets and apply pre-and postprossesing will be imported as follows:

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier

In [6]:
df_train_ini=pd.read_csv('https://s3.amazonaws.com/drivendata/data/7/public/4910797b-ee55-40a7-8668-10efd5c1b960.csv')
y_train=pd.read_csv('https://s3.amazonaws.com/drivendata/data/7/public/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv')
df_test_ini=pd.read_csv('https://s3.amazonaws.com/drivendata/data/7/public/702ddfc5-68cd-4d1d-a0de-f5f566f76d91.csv')

The training raw data contains `59400` records and `40` features. The testing dataset contains `14850` records. By inspecting each feature manually it is possible to tell the following story about the features:


### Summary of the features:
    
1. special features:

    - id: unique number refers to a certain pump.
    - construction_year: there are some zeros which are invalid data that should be replaced
    - num_private: there are some large values. they look like outliers. it contains large amount of 0s as invaild values
    - date_recorded: will change to datastamp as follows:

```{python}
df_train['date_recorded'] = pd.to_datetime(df_train['date_recorded'])
df_train['date_recorded']=df_train['date_recorded'].astype('int64')//1e9
```
    
2. numerical features:

    - amount_tsh: the amount of water: it contains 0s as invaild values. nonlinear transformation does not help to remove the 0 values
    - gps_height: the values look ok no invaild values (it not possible to tell if the 0s are invaild values)
    - longitude: there are some 0 values as invalid values and they should be replaced
    - latitude: No sign of invalid values
    - population: there are some zeros as invalid values

3. categorical features:

    - region_code: there are some outliers with log transformation the problem can be solved
    - district_code: there are some outliers but the log transform it can solved
    - funder: 3635 missing values, a lot of categories
    - installer: 3655 missing values, a lot of categories
    - wpt_name: a lot of categories there are some None values
    - payment: limited categories no sign for missing values
    - payment_type: limited categories no sign for missing values (there are some 0s)
    - water_quality:limited categories no sign for missing values (there are some 0s)
    - quality_group:limited categories no sign for missing values (there are some 0s)
    - source:limited categories no sign for missing values (there are some 0s)
    - source_type:limited categories no sign for missing values (there are some 0s) (high correlated with soruce)
    - source_class: alot of 0s-->unknown
    - waterpoint_type:limited categories no sign for missing values (there are some 0s)
    - waterpoint_type_group:limited categories no sign for missing values (there are some 0s)
    - basin:limited categories no sign for missing values
    - subvillage:a lot of categories there are no sign of missing values
    - region: limited categories no sign for missing values
    - lga:a lot of categories there are no sign of missing values
    - ward:a lot of categories there are no sign of missing values
    - public_meeting:3334 missing values:limited categories there are some signs for missing values (there are some 0s and -1)
    - recorded_by: one value no variation
    - scheme_management: 3877 missing values limited categories
    - scheme_name: 28166 missing values lot of categories
    - permit:3056 missing value two categories
    - extraction_type:limited categories no sign for missing values
    - extraction_type_group:limited categories no sign for missing values
    - extraction_type_class:limited categories no sign for missing values
    - management:limited categories no sign for missing values
    - management_group:limited categories no sign for missing values
    - quantity:limited categories no sign for missing values
    - quantity_group:limited categories no sign for missing values
    
### Labels:

There are three labels **status_group** that should be predicted:
1. functional
2. non functional
3. functional needs repair

In [65]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [66]:
features_all=list(df_train_ini.columns)
def f(feature_i):
    print(df_train_ini[features_all[feature_i]].head())  
interact(f, feature_i=widgets.IntSlider(min=0,max=39,step=1,value=0));

interactive(children=(IntSlider(value=0, description='feature_i', max=39), Output()), _dom_classes=('widget-interact',))

## Data cleaning:

To ensure the same processing on both the training and testing datasets both are merged in one dataset `df_total`. They will be splited again without any problems. The first step is to change the text values in categorical features to numerical values so the correlation between all features can be calculated.

In [67]:
frame=[df_train_ini,df_test_ini]
df_total=pd.concat(frame)

In [68]:
cat_f=['quantity','management_group','management','extraction_type_class','extraction_type_group',
       'extraction_type','permit','scheme_name','scheme_management','public_meeting','ward',
       'lga','region','subvillage','basin','waterpoint_type_group','waterpoint_type','source_class',
       'source_type','source','quality_group','water_quality','payment_type','payment','wpt_name','installer'
       ,'funder','quantity_group']

for i in cat_f:
    
    df_total[i]= df_total[i].astype('category').cat.codes

as mentioned before the date will be changed so it can be used as numerical values as follows:

In [69]:
df_total['date_recorded'] = pd.to_datetime(df_total['date_recorded'])
df_total['date_recorded']=df_total['date_recorded'].astype('int64')//1e9

In order to avoid outliers and some possible invalid values nonlinar transformation was applied as follows:

In [70]:
df_total['region_code']=np.log10(df_total['region_code']+1)
df_total['district_code']=np.log10(df_total['district_code']+1)
df_total['gps_height']=np.log10(df_total['gps_height']+100)
df_total['num_private']=np.log10(df_total['num_private']+1)
df_total['population']=np.log10(df_total['population']+1)
df_total['construction_year']=np.log10(df_total['construction_year']+1)

The histogram of the features can be explored as follows:

In [71]:
def g(feature_i):
    plt.hist(df_total[features_all[feature_i]], 50, normed=1, facecolor='green', alpha=0.75)
    plt.show() 
    
interact(g, feature_i=widgets.IntSlider(min=0,max=39,step=1,value=0));

interactive(children=(IntSlider(value=0, description='feature_i', max=39), Output()), _dom_classes=('widget-interact',))

In order to check the correlation between the features the following interaction code was used to show high correlated features:

In [81]:
def h(min_corr):
    corr_f=df_total.corr()
    corr_f[corr_f<min_corr/100]=0
    plt.matshow(corr_f)
    plt.show()
interact(h, min_corr=widgets.IntSlider(min=50,max=99,step=1,value=70));

interactive(children=(IntSlider(value=70, description='min_corr', max=99, min=50), Output()), _dom_classes=('widget-interact',))

Dropping columns that are high correlated with other columns or they have large amount of missing values:

In [42]:
df_total.drop(['id','recorded_by','source_type','waterpoint_type_group',
            'scheme_name','extraction_type_class','quantity_group'], axis = 1, inplace = True)

Resplit the total dataset after applying data cleaning:

In [43]:
df_total.index=range(0,len(df_total['funder']))    
df_train=df_total.loc[0:len(df_train_ini['funder'])-1,:]
df_test=df_total.loc[len(df_train_ini['funder']):,:]
df_train.index=df_train_ini.index
df_test.index=df_test_ini.index
len(df_test['funder'])

14850

## Model training:
In this step the training data will be splited to training and cross validation datasets as follows:
It is important to mention that the deep investigation that I did shows that the size of the cross validation set influences the accuracy significantly. The best score was obtained if the `test_size=0.01`.

In [44]:
X_train, X_cv, Y_train, y_cv= train_test_split(
        df_train,y_train['status_group'], test_size=0.01, random_state=42)

The classifier which based on `Random Forest` is defined as follows:
In this report only the best hyperparameters values after tuning using the `cross_val_score` that imported from `sklearn.model_selection import`.

In [45]:
clf_rf = RandomForestClassifier(n_estimators=600,max_depth=22,random_state=0)

The classifier defined in the step above is used to train the model as follows:

In [46]:
clf_rf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=22, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

The accuracy of the model on the training dataset can be calculated as follows:

In [47]:
print('train rf:',clf_rf.score(X_train, Y_train))

train rf: 0.976073870013


The accuracy of the model on the cross validation dataset can be calculated as follows:

In [49]:
print('cross validation rf:',clf_rf.score(X_cv, y_cv))

cross validation rf: 0.828282828283


## Predection

In this step the model developed above is used to predict the labels of the given testing dataset as follows:

In [50]:
y_sub=df_test_ini[['id']]
y_sub_rf=y_sub[['id']]
y_sub_rf['status_group']=clf_rf.predict(df_test)

Finally the prediction is saved in `csv` file to submit [here](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/)

In [51]:
y_sub_rf.to_csv('submission_MD_rf_best_score.csv', index=False)

My best score is `0.8249` and my ranking is `#42`